In [1]:
# import dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
from selenium import webdriver

# NASA Mars News

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.Mars_db
collection = db.articles
images = db.images

In [4]:
# open browser window
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# define website and open in browser window
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [6]:
# Retrieve the parent divs for all articles
for x in range(50):
    html = browser.html
    soup = bs(html, 'html.parser')

    articles = soup.find_all(class_='list_text')
    print(articles)

ill Reach the Red Planet's Surface</a></div><div class="article_teaser_body">The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.</div></div>, <div class="list_text"><div class="list_date">June 17, 2020</div><div class="content_title"><a href="/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/" target="_self">The Launch Is Approaching for NASA's Next Mars Rover, Perseverance</a></div><div class="article_teaser_body">The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. </div></div>, <div class="list_text"><div class="list_date">June 15, 2020</div><div class="content_title"><a href="/news/8692/nasa-to-hold-mars-2020-perseverance-rover-launch-briefing/" target="_self">NASA to Hold Mars 2020 Perseverance Rover Launch Briefing</a></div><div class=

In [ ]:
# Close the browser after scraping
browser.quit()

In [7]:
# loop over results to get article data
for article in articles:
    # scrape the article title
    news_title = article.find(class_='content_title').text
    # scrape the article paragraph
    news_p = article.find(class_='article_teaser_body').text
    # print article data
    print('------------')
    print(news_title)
    print(news_p)
    #dictionary to be inserted into MongoDB for later retrieval
    post = {
        'Title': news_title,
        'Paragraph': news_p
    }
    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)

------------
Hear Audio From NASA's Perseverance As It Travels Through Deep Space
The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.
------------
Mars Is Getting a New Robotic Meteorologist
Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.
------------
Heat and Dust Help Launch Martian Water Into Space, Scientists Find
Scientists using an instrument aboard NASA’s Mars Atmosphere and Volatile EvolutioN, or MAVEN, spacecraft have discovered that water vapor near the surface of the Red Planet is lofted higher into the atmosphere than anyone expected was possible. 
------------
NASA's Curiosity Takes Selfie With 'Mary Anning' on the Red Planet
The Mars rover has drilled three samples of rock in this clay-enriched region since arriving in July.
------------
Independent Review Indicates NASA Prepared f

In [45]:
# Display items in MongoDB collection
list = db.articles.find()

for listing in list:
    print(listing)

{'_id': ObjectId('5fb8c5ed8f43340cd298dcd1'), 'Title': "Hear Audio From NASA's Perseverance As It Travels Through Deep Space", 'Paragraph': "The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight."}
{'_id': ObjectId('5fb8c5ed8f43340cd298dcd2'), 'Title': 'Mars Is Getting a New Robotic Meteorologist', 'Paragraph': "Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles."}
{'_id': ObjectId('5fb8c5ed8f43340cd298dcd3'), 'Title': 'Heat and Dust Help Launch Martian Water Into Space, Scientists Find', 'Paragraph': 'Scientists using an instrument aboard NASA’s Mars Atmosphere and Volatile EvolutioN, or MAVEN, spacecraft have discovered that water vapor near the surface of the Red Planet is lofted higher into the atmosphere than anyone expected was possible. '}
{'_id': ObjectId('5fb8c5ed8f43340cd298dcd4'), 'Title': "

# JPL Mars Space Images - Feature Image

In [108]:
# open browser window
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [109]:
# define website and open in browser window
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [110]:
# get to the page with the image url
browser.links.find_by_partial_text('FULL IMAGE').first.click()
browser.links.find_by_partial_text('more info').first.click()
html = browser.html
soup = bs(html, 'html.parser')
# Identify and return the images
results = soup.select_one('figure.lede a img').get('src')

# Close the browser after scraping
browser.quit()

# store the image url
image_url = f'https://www.jpl.nasa.gov{results}'

# Url to be inserted as a MongoDB document
post = {
    'feature_image_url': image_url
}
images.insert_one(post)

# Mars Facts

In [4]:
url = 'https://space-facts.com/mars/'

# Use Panda's `read_html` to parse the url
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [5]:
# extract the table containing facts about Mars
mars_fact_df = tables[0]
mars_fact_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [6]:
# rename the columns
mars_fact_df = mars_fact_df.rename(columns={0: 'Description', 1: 'Mars_planet'})
mars_fact_df = mars_fact_df.set_index('Description')
mars_fact_df

,Mars_planet
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [8]:
# export table to HTML format
Mars_table = mars_fact_df.to_html('static/Mars_table.html')
Mars_table

# Mars Hemispheres

## Cerberus

In [142]:
# open browser window
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [143]:
# define website and open in browser window
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [144]:

# get to the page with the image url
# another option:
# browser.find_by_css('h3')[0].click()
browser.links.find_by_partial_text('Cerberus Hemisphere').first.click()

html = browser.html
soup = bs(html, 'html.parser')
# Identify and return the images
results = soup.select_one('img.wide-image').get('src')
print(results)

# Close the browser after scraping
browser.quit()

# store the image url
image_url = f'https://astrogeology.usgs.gov{results}'

# Url to be inserted as a MongoDB document
post = {
    'cerberus_image_url': image_url
}
images.insert_one(post)



/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg


## Schiaparelli

In [145]:
# open browser window
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [146]:
# define website and open in browser window
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [147]:
# get to the page with the image url
browser.links.find_by_partial_text('Schiaparelli Hemisphere').first.click()

html = browser.html
soup = bs(html, 'html.parser')
# Identify and return the images
results = soup.select_one('img.wide-image').get('src')
print(results)

# Close the browser after scraping
browser.quit()

# store the image url
image_url = f'https://astrogeology.usgs.gov{results}'

# Url to be inserted as a MongoDB document
post = {
    'schiaparelli_image_url': image_url
}
images.insert_one(post)


/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg


## Syrtis Major

In [148]:
# open browser window
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [149]:
# define website and open in browser window
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [150]:
# get to the page with the image url
browser.links.find_by_partial_text('Syrtis Major Hemisphere').first.click()

html = browser.html
soup = bs(html, 'html.parser')
# Identify and return the images
results = soup.select_one('img.wide-image').get('src')
print(results)

# Close the browser after scraping
browser.quit()

# store the image url
image_url = f'https://astrogeology.usgs.gov{results}'

# Url to be inserted as a MongoDB document
post = {
    'syrtis_major_image_url': image_url
}
images.insert_one(post)

/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg


## Valles Marineris

In [151]:
# open browser window
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [152]:
# define website and open in browser window
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [153]:
# get to the page with the image url
browser.links.find_by_partial_text('Valles Marineris Hemisphere').first.click()

html = browser.html
soup = bs(html, 'html.parser')
# Identify and return the images
results = soup.select_one('img.wide-image').get('src')
print(results)

# Close the browser after scraping
browser.quit()

# store the image url
image_url = f'https://astrogeology.usgs.gov{results}'

# Url to be inserted as a MongoDB document
post = {
    'valles_marineris_image_url': image_url
}
images.insert_one(post)

/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
